In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Quantum Linear Regression

In [6]:
df = pd.read_csv('cleaned_quantum.csv')

In [7]:
X = df.drop(columns=['primary_energy_consumption'])
y = df['primary_energy_consumption']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def quantum_feature_map(X, gamma=1.0):
    X = np.asarray(X)
    return np.concatenate([np.cos(gamma * X), np.sin(gamma * X)], axis=1)

X_train_mapped = quantum_feature_map(X_train, gamma=0.21)
X_test_mapped = quantum_feature_map(X_test, gamma=0.21)

In [8]:
model = LinearRegression()
model.fit(X_train_mapped, y_train)

LinearRegression()

In [9]:
y_pred = model.predict(X_test_mapped)
r2 = r2_score(y_test, y_pred)
print(f"Quantum Linear Regression R2 Score: {r2:.4f}")

Quantum Linear Regression R2 Score: 0.2133


In [10]:
import pickle

In [11]:
with open("quantum_linear_regression.pkl", "wb") as f:
    pickle.dump(model, f)

# Quantum Kernel Ridge Regression

In [12]:
def quantum_feature_map(x, gamma=0.01):
    x = np.asarray(x)
    return np.concatenate([np.cos(gamma * x), np.sin(gamma * x)])

def quantum_kernel(X1, X2, gamma=0.01):
    K = np.zeros((len(X1), len(X2)))
    for i in range(len(X1)):
        for j in range(len(X2)):
            phi_x = quantum_feature_map(X1[i], gamma)
            phi_y = quantum_feature_map(X2[j], gamma)
            K[i, j] = np.dot(phi_x, phi_y)
    return K

### Code for basic quantum kernelRidge

In [13]:
from sklearn.kernel_ridge import KernelRidge

In [14]:
X = df.drop(columns=['primary_energy_consumption'])
y = df['primary_energy_consumption']

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

In [15]:
gamma = 0.1

X_train_small = X_train[:1000]
X_test_small = X_test[:200]
K_train = quantum_kernel(X_train_small, X_train_small, gamma=0.1)
K_test = quantum_kernel(X_test_small, X_train_small, gamma=0.1)

In [16]:
model = KernelRidge(alpha=1.0, kernel='precomputed')
model.fit(K_train, y_train[:1000])

KernelRidge(alpha=1.0, kernel='precomputed')

In [30]:
y_pred = model.predict(K_test)

r2 = r2_score(y_test[:200], y_pred)
print(f"Quantum-Inspired Kernel Ridge Regression R2 Score (gamma={gamma}): {r2:.4f}")

Quantum-Inspired Kernel Ridge Regression R2 Score (gamma=0.1): 0.3452


### Trying differrent aplha and gamma values

In [31]:
from sklearn.model_selection import GridSearchCV

target_col = 'primary_energy_consumption'  

alphas = [0.01, 0.1, 1.0, 10.0]
gammas = [0.01, 0.05, 0.1, 0.5]

best_score = -999
best_combo = (None, None)

for alpha in alphas:
    for gamma in gammas:
        K_train = quantum_kernel(X_train_small, X_train_small, gamma=gamma)
        K_test = quantum_kernel(X_test_small, X_train_small, gamma=gamma)

        model = KernelRidge(alpha=alpha, kernel='precomputed')
        model.fit(K_train, y_train[:1000])
        y_pred = model.predict(K_test)

        score = r2_score(y_test[:200], y_pred)
        if score > best_score:
            best_score = score
            best_combo = (alpha, gamma)

print(f"Best R2 Score: {best_score:.4f} with alpha={best_combo[0]} and gamma={best_combo[1]}")

Best R2 Score: 0.8736 with alpha=0.01 and gamma=0.01


### Training the entire dataset

In [34]:
target_col = 'primary_energy_consumption'

X_full = df.drop(columns=[target_col]).values
y_full = df[target_col].values

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

K_train = quantum_kernel(X_train, X_train, gamma=0.01)
K_test = quantum_kernel(X_test, X_train, gamma=0.01)

model = KernelRidge(alpha=0.01, kernel='precomputed')
model.fit(K_train, y_train)

y_pred = model.predict(K_test)

r2 = r2_score(y_test, y_pred)
print(f"R2 Score on test data: {r2:.4f}")

R2 Score on test data: 0.8754


In [35]:
with open("quantum_kernel_ridge_regression.pkl", "wb") as f:
    pickle.dump(model, f)